# Import

In [5]:
import pandas as pd
import difflib

# Data Preparation

In [6]:
left_data = pd.read_csv("C:/Users/WHR/Desktop/final_project/group-project-spring-2024/Data/GroupProject/entity-resolution/left_dataset.csv")
right_data = pd.read_csv("C:/Users/WHR/Desktop/final_project/group-project-spring-2024/Data/GroupProject/entity-resolution/right_dataset.csv")

In [7]:
left = pd.DataFrame(left_data)
right = pd.DataFrame(right_data)

In [8]:
print("left", len(left))
print("right", len(right))

left 94585
right 91791


In [9]:
for col in left.columns:
    left[col] = left[col].apply(lambda x: x.lower() if type(x) == str else x)
for col in right.columns:
    right[col] = right[col].apply(lambda x: x.lower() if type(x) == str else x)

In [10]:
cities_left_count = left['city'].nunique()
cities_right_count = right['city'].nunique()
print(cities_left_count, cities_right_count)

794 44


In [11]:
cities_left = left['city'].unique()
left_list = cities_left.tolist()

cities_right = right['city'].unique()
right_list = cities_right.tolist()

# Processing

In [13]:
def find_matches(df1, df2, key1_primary, key2_primary, key1_secondary, key2_secondary, threshold=0.8):
    matches = []
    for i, row1 in df1.iterrows():
        best_score = threshold
        best_match = None
        for j, row2 in df2.iterrows():
            primary1 = str(row1[key1_primary]) if pd.notna(row1[key1_primary]) else ''
            primary2 = str(row2[key2_primary]) if pd.notna(row2[key2_primary]) else ''
            score = difflib.SequenceMatcher(None, primary1, primary2).ratio()
            if score > best_score:
                best_score = score
                best_match = (row1['entity_id'], row2['business_id'], score)
        if best_match:
            matches.append(best_match)
        else:
            secondary1 = str(row1[key1_secondary]) if pd.notna(row1[key1_secondary]) else ''
            secondary2 = str(row2[key2_secondary]) if pd.notna(row2[key2_secondary]) else ''
            score = difflib.SequenceMatcher(None, secondary1, secondary2).ratio()
            if score > best_score:
                best_score = score
                best_match = (row1['entity_id'], row2['business_id'], score)
            if best_match:
                matches.append(best_match)
    return matches

In [14]:
def match_all(df1, df2, city_list):
    
    all_matches = []
    
    for city in city_list:
        
        left_data = df1[df1['city'] == city]
        right_data = df2[df2['city'] == city]
        
        refined_matches = find_matches(left_data, right_data, 'name', 'name', 'address', 'address')
        all_matches.append(refined_matches)
        
    return all_matches

In [15]:
result = match_all(left, right, right_list)

In [16]:
len(result)

44

In [20]:
count = 0
for match in result:
    count += len(match)
print(count)

6517
